In [1]:
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

from gensim import corpora
from gensim import models
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()

from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.utils import pprint

import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel

from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
font_loaction = 'C:/Users/User/Desktop/NANUMGOTHIC.ttf'
import warnings
warnings.filterwarnings('ignore')


c:\Users\User\anaconda3\envs\version_test\lib\site-packages\ipykernel_launcher.py:14: MatplotlibDeprecationWarning: 
The get_fontconfig_fonts function was deprecated in Matplotlib 3.5 and will be removed two minor releases later.
  
c:\Users\User\anaconda3\envs\version_test\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [48]:
category = ['음식료품', '섬유의복', '종이목재', '화학', '의약품', '비금속광물', '철강금속', '기계',
        '전기전자', '의료정밀', '운수장비', '유통업', '전기가스업', '건설업', '운수창고', '통신업', '금융업',
        '은행', '증권', '보험', '서비스업']

# category = ['음식료품' ,'섬유의복', '종이목재']
ddf = pd.read_csv("C:/Users/user/github/Web_Crawling/save_data/cyjhword.csv")
hannanum = Hannanum()
kkma = Kkma()

In [49]:
stop_words = ddf['불용어'].values.tolist()
negative_words = ddf['부정어'].dropna().values.tolist()
positive_words = ddf['긍정어'].dropna().values.tolist()

In [50]:
# 1차 불용어 처리
result = pd.DataFrame()
for j in tqdm(range(len(category))):
    name = category[j]
    df = pd.read_csv('C:/Users/user/github/Web_Crawling/save_data/'+'{}'.format(name)+'_total.csv') # 데이터 불러오기
    count_positive = 0
    count_negative = 0
    year_list = []
    month_list = []
    df['{}_positive'.format(category[j])] = 0
    df['{}_negative'.format(category[j])] = 0
    line = []
    df['time'] = df['time'].astype(str)
    df['time']=pd.to_datetime(df.iloc[:,0])

    for i in tqdm(range(len(df))):
        count_positive = 0
        count_negative = 0
        year_list.append(df.iloc[i,0].year)
        month_list.append(df.iloc[i,0].month)
        a = df.iloc[:,1][i] # 모든 title은 1행에 존재
        #h = hannanum.nouns(a) # 형태 분석
        #k = kkma.nouns(a)
        k1 = kkma.pos(a)
        for (k ,tclass) in k1:
            if (tclass == 'NNM' or tclass == 'NNG' or tclass=='NNP' or tclass=='NNB' or tclass=='NP') & (len(k)>1):
                line.append(k)
        z = list(set(line) - set(stop_words))
        for b in z:
            if b in positive_words:
                count_positive += 1
            if b in negative_words:
                count_negative += 1
        df.iloc[i, 3] = count_positive
        df.iloc[i, 4] = count_negative
    df['year'] = year_list
    df['month'] = month_list
    if j==0:
        first = df.groupby(['year','month'])['{}_negative'.format(category[0]),'{}_positive'.format(category[0])].sum()
        first['{}'.format(category[j])+'_negative_rate'] = first.iloc[:,0] / (first.iloc[:,0] + first.iloc[:,1])
        first['{}'.format(category[j])+'_positive_rate'] = first.iloc[:,1] / (first.iloc[:,0] + first.iloc[:,1])
        result = pd.concat([result, first], axis=1)
        add  = pd.DataFrame()
    else:
        add = df.groupby(['year','month'])['{}_negative'.format(category[j]),'{}_positive'.format(category[j])].sum()
        add['{}'.format(category[j])+'_negative_rate'] = add.iloc[:,0] / (add.iloc[:,0] + add.iloc[:,1])
        add['{}'.format(category[j])+'_positive_rate'] = add.iloc[:,1] / (add.iloc[:,0] + add.iloc[:,1])
    result = pd.concat([result,add],axis=1)  
    
#     save = pd.DataFrame(z) # 저장할 df만들기
#     save.to_csv('C:/Users/user/github/Web_Crawling/csv/'+'{}'.format(name)+'_token.csv', encoding='utf-8-sig',index=False)
#     dictionary = corpora.Dictionary(z)                    # 사전 생성 (토큰화)

#     corpus = [dictionary.doc2bow(text) for text in z]     # 말뭉치 생성 (벡터화)
#     num_topics = 5
#     chunksize = 2000
#     passes = 20
#     iterations = 400
#     eval_every = None

#     temp = dictionary[0]
#     id2word = dictionary.id2token

#     model = LdaModel(
#         corpus=corpus,
#         id2word=id2word,
#         chunksize=chunksize,
#         alpha='auto',
#         eta='auto',
#         iterations=iterations,
#         num_topics=num_topics,
#         passes=passes,
#         eval_every=eval_every
#         )
#     lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
#     pyLDAvis.save_html(lda_visualization,'{}'.format(name) + 'hannanum.html')
result.to_csv('C:/Users/user/github/Web_Crawling/save_data/total_count1.csv', encoding='cp949',index=True)

  0%|          | 0/21 [00:00<?, ?it/s]